In [178]:
import h2o
import numpy as np
import pandas as pd

In [179]:
diamonds =  pd.read_csv('./diamonds-datamad1019/data.csv/data.csv')

In [180]:
color = {
    "D": 7,
    "E": 6,
    "F": 5,
    "G": 4,
    "H": 3,
    "I": 2,
    "J": 1
}

cut = {
    "Ideal": 5,
    "Premium": 4,
    "Very Good": 3,
    "Good": 2,
    "Fair":1
}

clarity = {
    "IF": 8,
    "VVS1": 7,
    "VVS2": 6,
    "VS1": 5,
    "VS2": 4,
    "SI1": 3,
    "SI2": 2,
    "I1": 1
}

In [181]:
for i in range(len(diamonds)):
    co = color[diamonds.loc[i, "color"]]
    cu = cut[diamonds.loc[i, "cut"]]
    cl = clarity[diamonds.loc[i, "clarity"]]
    diamonds.loc[i, "color_num"] = co
    diamonds.loc[i, "cut_num"] = cu
    diamonds.loc[i, "clarity_num"] = cl

In [182]:
diamonds.drop(columns=['cut','color','clarity'], inplace=True)
diamonds['volume'] = diamonds['x']*diamonds['y']*diamonds['z']
diamonds.drop(['x','y','z'],axis=1,inplace=True)

In [183]:
diamonds

,carat,depth,table,price,color_num,cut_num,clarity_num,volume
0,2.26,61.9,57.0,12831,4.0,5.0,2.0,366.903680
1,2.43,63.2,57.0,16170,3.0,3.0,2.0,392.176400
2,0.80,61.0,57.0,2797,5.0,4.0,2.0,133.001901
3,0.40,63.3,60.0,630,5.0,5.0,1.0,64.059840
4,0.31,61.6,55.0,698,4.0,5.0,4.0,51.797610
...,...,...,...,...,...,...,...,...
40450,1.11,62.8,61.0,5315,3.0,4.0,3.0,180.060192
40451,0.73,62.6,56.0,2762,5.0,5.0,4.0,119.231280
40452,1.26,59.2,60.0,6855,2.0,3.0,5.0,208.046124
40453,0.72,61.4,56.0,2297,4.0,5.0,2.0,119.547648


In [137]:
h2o.init(nthreads = -1, max_mem_size = 26)

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 2 mins
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_mariobru_4n4h1g
H2O cluster total nodes:,1
H2O cluster free memory:,22.78 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [184]:
diamonds_h2o=h2o.H2OFrame(diamonds)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [185]:
diamonds_h2o.head()

carat,depth,table,price,color_num,cut_num,clarity_num,volume
2.26,61.9,57,12831,4,5,2,366.904
2.43,63.2,57,16170,3,3,2,392.176
0.8,61,57,2797,5,4,2,133.002
0.4,63.3,60,630,5,5,1,64.0598
0.31,61.6,55,698,4,5,4,51.7976
0.53,60.5,60,1132,5,3,2,85.5027
0.7,60.8,61,1987,7,4,2,110.716
1.13,64,58,3952,5,2,2,179.758
0.35,60.9,55,614,6,5,3,57.3448
1.01,61.4,66,5062,3,1,5,153.103


In [186]:
y_columns = "price"
x_columns = ["carat","color_num","cut_num","clarity_num","depth"]

In [187]:
train, test=diamonds_h2o.split_frame(ratios = [.8])
X_train=train[x_columns]
y_train=train[y_columns]
X_test=test[x_columns]
y_test=test[y_columns]

In [188]:
from h2o.automl import H2OAutoML
aml_ti = H2OAutoML(max_runtime_secs= 180,max_models= 15, seed= 1, nfolds=0)
aml_ti.train(x = x_columns, y = y_columns, training_frame = train, validation_frame=test)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [189]:
lb_ti = aml_ti.leaderboard
lb_ti

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_1_AutoML_20191209_135213,319693,565.414,319693,293.579,0.114556
XGBoost_1_AutoML_20191209_135213,322496,567.887,322496,287.981,0.105622
XGBoost_3_AutoML_20191209_135213,328277,572.955,328277,299.554,0.118841
GBM_3_AutoML_20191209_135213,331734,575.964,331734,293.025,0.109414
GBM_2_AutoML_20191209_135213,331929,576.133,331929,295.178,0.112426
XGBoost_2_AutoML_20191209_135213,332384,576.528,332384,293.114,0.10929
GBM_5_AutoML_20191209_135213,332785,576.876,332785,293.147,0.10781
GBM_4_AutoML_20191209_135213,337598,581.031,337598,294.669,0.108885
XGBoost_grid_1_AutoML_20191209_135213_model_1,369610,607.956,369610,303.109,0.108562
DRF_1_AutoML_20191209_135213,372539,610.36,372539,311.985,0.120238


In [190]:
pred_automl = aml_ti.leader.predict(test)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [191]:
pred_automl

predict
662.987
4494.48
679.636
643.333
2922.28
852.177
710.269
671.012
11479.9
869.73


In [192]:
y_test

price
698
3952
614
552
2775
741
596
727
11565
825


In [193]:
#Probamos con test

In [194]:
diamonds_test =  pd.read_csv('./diamonds-datamad1019/test.csv')

In [195]:
for i in range(len(diamonds_test)):
    co = color[diamonds_test.loc[i, "color"]]
    cu = cut[diamonds_test.loc[i, "cut"]]
    cl = clarity[diamonds_test.loc[i, "clarity"]]
    diamonds_test.loc[i, "color_num"] = co
    diamonds_test.loc[i, "cut_num"] = cu
    diamonds_test.loc[i, "clarity_num"] = cl

In [196]:
diamonds_test.drop(columns=['cut','color','clarity'], inplace=True)

In [197]:
diamonds_test['volume'] = diamonds_test['x']*diamonds_test['y']*diamonds_test['z']
diamonds_test.drop(['x','y','z'],axis=1,inplace=True)

In [198]:
diamonds_h2o=h2o.H2OFrame(diamonds_test)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [199]:
x_columns = ["carat","color_num","cut_num","clarity_num","depth"]

In [200]:
X_train=diamonds_h2o[x_columns]

In [201]:
pred_automl = aml_ti.leader.predict(X_train)

gbm prediction progress: |████████████████████████████████████████████████| 100%


In [202]:
pred_automl

predict
1358.16
6572.92
1662.38
4007.66
1803.2
9629.6
790.261
899.943
702.668
14080.4


In [203]:
submission = pred_automl.as_data_frame()

In [204]:
submission

,predict
0,1358.155372
1,6572.915062
2,1662.377156
3,4007.664219
4,1803.203048
...,...
13480,2167.678827
13481,2150.039667
13482,11314.476398
13483,525.726962


In [205]:
submission.rename(columns={"predict": "price"}, inplace=True)
submission

,price
0,1358.155372
1,6572.915062
2,1662.377156
3,4007.664219
4,1803.203048
...,...
13480,2167.678827
13481,2150.039667
13482,11314.476398
13483,525.726962


In [206]:
submission.price = submission.price.astype(int)


In [207]:
submission

,price
0,1358
1,6572
2,1662
3,4007
4,1803
...,...
13480,2167
13481,2150
13482,11314
13483,525


In [208]:
submission.to_csv('./diamonds-datamad1019/submission.csv')